In [89]:
import random
import requests
import pandas as pd
import re
import time

In [90]:
anime_list = []
for x in range(1,251,1):
    if x != 1:
        time.sleep(random.randint(5,10))
    response = requests.get(f'https://api.jikan.moe/v4/top/anime?page={x}')
    
    json_data = response.json()
    
    # json data consists of 'pagination' and 'data'
    for entry in json_data['data']:
        
        # Find airing dates
        try:
            from_date = pd.Timestamp(year=entry['aired']['prop']['from']['year'],month=entry['aired']['prop']['from']['month'],day=entry['aired']['prop']['from']['day'])
        except TypeError:
            from_date = pd.NaT
        
        try:
            to_date = pd.Timestamp(year=entry['aired']['prop']['to']['year'],month=entry['aired']['prop']['to']['month'],day=entry['aired']['prop']['to']['day'])
        except TypeError:
            to_date = pd.NaT
        
        # Find duration    
        duration = 0;
        hours = re.findall(r'\d+ ? h', entry['duration'])
        if len(hours) > 0:
            converted_hours = re.findall(r'\d+',hours[0])
            if len(converted_hours) > 0:
                duration =+ int(converted_hours[0])*60
                
        minutes = re.findall(r'\d+ ?m', entry['duration'])
        if len(minutes) > 0:
            found_minutes = re.findall(r'\d+',minutes[0])
            if len(found_minutes) > 0:
                duration+= int(found_minutes[0])
        
        # Create list for genres
        genre_list = []
        for genre in entry['genres']:
            genre_list.append(genre['name'])
            
        # Create list for themes
        theme_list = []
        for theme in entry['themes']:
            theme_list.append(theme['name'])
            
        # Create list for demographic
        demographic_list = []
        for demographic in entry['demographics']:
            demographic_list.append(demographic['name'])
            
            
        anime = [entry['titles'][0]['title'], entry['type'], entry['source'], entry['episodes'],
                 entry['status'], from_date, to_date,duration,entry['rating'],entry['score'],entry['scored_by'],
                 entry['rank'],entry['popularity'],entry['members'],entry['favorites'],entry['season'],
                 entry['year'],genre_list,theme_list,demographic_list]
        
        anime_list.append(anime)

In [91]:
headers = ['Title','Type','Source','Episodes','Status','Started At','Ended At','Duration in Minutes','Rating','Score','Scored By',
           'Score Rank','Popularity Rank','Members','Favorites','Season','Year','Genre','Theme','Demographic']
df = pd.DataFrame(anime_list,columns=headers)
df

,Title,Type,Source,Episodes,Status,Started At,Ended At,Duration in Minutes,Rating,Score,Scored By,Score Rank,Popularity Rank,Members,Favorites,Season,Year,Genre,Theme,Demographic
0,Sousou no Frieren,TV,Manga,28.0,Currently Airing,2023-09-29,2024-03-22,24,PG-13 - Teens 13 or older,9.15,189697,1.0,379,566535,20720,fall,2023.0,"[Adventure, Drama, Fantasy]",[],[Shounen]
1,Fullmetal Alchemist: Brotherhood,TV,Manga,64.0,Finished Airing,2009-04-05,2010-07-04,24,R - 17+ (violence & profanity),9.09,2100761,2.0,3,3312847,224338,spring,2009.0,"[Action, Adventure, Drama, Fantasy]",[Military],[Shounen]
2,Steins;Gate,TV,Visual novel,24.0,Finished Airing,2011-04-06,2011-09-14,24,PG-13 - Teens 13 or older,9.07,1387448,3.0,13,2540312,188400,spring,2011.0,"[Drama, Sci-Fi, Suspense]","[Psychological, Time Travel]",[]
3,Gintama°,TV,Manga,51.0,Finished Airing,2015-04-08,2016-03-30,24,PG-13 - Teens 13 or older,9.06,248787,4.0,338,624445,16538,spring,2015.0,"[Action, Comedy, Sci-Fi]","[Gag Humor, Historical, Parody, Samurai]",[Shounen]
4,Shingeki no Kyojin Season 3 Part 2,TV,Manga,10.0,Finished Airing,2019-04-29,2019-07-01,23,R - 17+ (violence & profanity),9.05,1567159,5.0,21,2246431,58151,spring,2019.0,"[Action, Drama, Suspense]","[Gore, Military, Survival]",[Shounen]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6245,Choujikuu Seiki Orguss 02,OVA,Original,6.0,Finished Airing,1993-12-05,1995-04-25,30,PG-13 - Teens 13 or older,6.69,1259,5635.0,9113,3851,11,None,NaN,"[Action, Adventure, Romance, Sci-Fi]",[Mecha],[]
6246,Shin Don Chuck Monogatari,TV,Unknown,73.0,Finished Airing,1976-04-07,1978-03-25,25,G - All Ages,6.69,365,5682.0,13984,824,3,spring,1976.0,"[Adventure, Slice of Life]",[],[Kids]
6247,Cutie Honey F,TV,Manga,39.0,Finished Airing,1997-02-15,1998-01-31,23,PG-13 - Teens 13 or older,6.69,2570,5636.0,6246,10252,34,winter,1997.0,"[Action, Adventure, Comedy, Romance, Sci-Fi, E...",[],[Shoujo]
6248,Sabu to Ichi Torimono Hikae,TV,Manga,52.0,Finished Airing,1968-10-03,1969-09-24,25,R - 17+ (violence & profanity),6.69,374,5665.0,10363,2551,9,fall,1968.0,"[Action, Adventure, Drama, Slice of Life]","[Detective, Historical, Martial Arts, Samurai]",[Shounen]


In [92]:
df.to_csv('~/PycharmProjects/MAL_Scraper/anime_list.csv')